In [31]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

def createAvitoSearchPageUrl(category,searchQuery,page):
    domain = u'https://www.avito.ru/'
    searchQuery = searchQuery.replace(u' ',u'+')
    url = u'{0}{1}?q={2}&sgtd=2&p={3}'.format(domain,category,searchQuery,page)
    return url
def getSoupByUrl(url):
    proxies = {'http': 'http://94.77.88.2:3001',
          'https': 'https://94.77.88.2:3001'}
    return BeautifulSoup(requests.get(url, proxies=proxies).text,'lxml')
def checkIfPageIsNotContainsResults(soup):
    return u'ничего не найдено' in ''.join([tag.get_text() for tag in soup.find_all(attrs={'class':'nulus'})])
def getAdsFromAvitoSearchPage(soup):
    return soup.find(attrs={'class':'catalog-list'}).find_all(attrs={'class':'description'})
def getAdParams(soup):
    price = soup.find(attrs={'class':'about'}).get_text().replace('\n','').strip()
    shortAdData = soup.find(attrs={'class':'data'}).get_text().replace('\n','').strip()
    header = soup.find('h3').a['title']
    link = soup.find('h3').a['href']
    return {'header':header,'link':link,'shortAdData':shortAdData,'price':price,'link':link}
def getAdParamsFromAvitoSearchPage(ads):
    return [getAdParams(ad) for ad in ads]
def createAvitoAdPageUrl(link):
    domain = u'https://www.avito.ru'
    return '{0}{1}'.format(domain,link)
def getAdExtendedParamsBySoup(soup):
    todayDate = datetime.now().isoformat()
    header = soup.find('h1').get_text()
    placementDate = soup.find(attrs={'class':'item-subtitle'}).get_text().split('\n')[1].strip()
    price = soup.find(attrs={'itemprop':'price'}).get_text().strip()
    seller = soup.find(attrs={'id':'seller'}).get_text().strip().replace('\n','')
    city = soup.find(attrs={'itemprop':'availableAtOrFrom'}).find(attrs={'class':'description_content'}).get_text().replace('\n','').strip()
    descriptionText = soup.find(id='desc_text').get_text()
    adId = soup.find(id='item_id').get_text()
    return {'todayDate':todayDate,
            'header':header,
            'placementDate':placementDate,
            'price':price,
            'seller':seller,
            'city':city,
            'descriptionText':descriptionText,
            'adId':adId
           }
def getExtendedAdsParamsByListOfAds(ads):
    extendedAdsParams = []
    for ad in ads:
        adParams = getAdParams(ad)
        print adParams['link']
        try:
            soup = getSoupByUrl(createAvitoAdPageUrl(adParams['link']))
            extendedAdParams = getAdExtendedParamsBySoup(soup)
            extendedAdParams['link'] = adParams['link']
            extendedAdsParams.append(extendedAdParams)
        except:
            continue
    return extendedAdsParams

# category = u'moskovskaya_oblast/predlozheniya_uslug/transport_perevozki'
# query = u'грузовые перевозки'
# page = 1
# soup = getSoupByUrl(createAvitoSearchPageUrl(category,query,page))
# if checkIfPageIsNotContainsResults(soup):
#     ads = getAdsFromAvitoSearchPage(soup)
# allExtendedAdsParams = getExtendedAdsParamsByListOfAds(ads)
# allShortAdParams = []
# for ad in ads:
#     allShortAdParams.append(getAdParams(ad))
# df = pd.DataFrame(allShortAdParams)
# df.merge(pd.DataFrame(allExtendedAdsParams),on='link',how='outer').to_excel('D:\\avito_ads.xlsx')

In [12]:
searchQueries = []
with open(u'D:\\Проект Горячие лиды\\spisok_zaprosov.txt','r+') as f1:
    for row in f1:
            searchQueries.append(row.replace('\n','').lower().decode('utf-8'))

In [2]:
def getAvitoQueryDataFrame(query):
    category = u'moskovskaya_oblast/predlozheniya_uslug/transport_perevozki'
    dataframe = pd.DataFrame()
    for page in xrange(1,10):
        print page
        soup = getSoupByUrl(createAvitoSearchPageUrl(category,query,page))
        if not checkIfPageIsNotContainsResults(soup):
            print 'ok'
            ads = getAdsFromAvitoSearchPage(soup)
            allExtendedAdsParams = getExtendedAdsParamsByListOfAds(ads)
            allShortAdParams = []
            for ad in ads:
                allShortAdParams.append(getAdParams(ad))
            df = pd.DataFrame(allShortAdParams)
            df = df.merge(pd.DataFrame(allExtendedAdsParams),on='link',how='outer')
            dataframe = pd.concat([dataframe,df])
    return dataframe

In [32]:
getAvitoQueryDataFrame(u'откачка септиков')

1
ok
/iksha/predlozheniya_uslug/otkachka_septikov_i_kanalizatsii_657896736
/solnechnogorsk/predlozheniya_uslug/otkachka_septikov_vygrebnyh_yam_i_kanalizatsii_202463413
/schelkovo/predlozheniya_uslug/moskva_oblast_bort_20_tonn_500571398
/golitsyno/predlozheniya_uslug/ilosos_dlya_otkachki_bentonita_gnb_shlam_733885377
/odintsovo/predlozheniya_uslug/otkachka_vygrebnyh_yam._septikov_709651661
/nekrasovskiy/predlozheniya_uslug/otkachka_kanalizatsii_657921227
/lobnya/predlozheniya_uslug/otkachka_vygrebnyh_yam_septikov_kanalizatsii_657931762
/istra/predlozheniya_uslug/otkachka_kanalezatsii_septikov_biotualetov_istra_5m3_822228171
/golitsyno/predlozheniya_uslug/otkachka_septikov_vygrebnyh_yam_tualetov_zvonite_672220227
/elektrougli/predlozheniya_uslug/otkachka_kanalizatsii_septikov_780412666
/balashiha/predlozheniya_uslug/otkachka_kanalizatsii_septikov_780416968
/korolev/predlozheniya_uslug/assenizator_822097018
/noginsk/predlozheniya_uslug/otkachka_chistka_dna_ustranenie_zasorov_372774915
/is

,header_x,link,price_x,shortAdData,adId,city,descriptionText,header_y,placementDate,price_y,seller,todayDate
0,Откачка септиков и канализации в Икше,/iksha/predlozheniya_uslug/otkachka_septikov_i...,1 500 руб.,"Транспорт, перевозки Частное лицо ...",657896736,"Московская область, Икша","Откачка септиков, выгребных ям.Объем машины 4 ...",Откачка септиков и канализации,Размещено сегодня в 15:16.,1 500 руб.,Роман (частное лицо) на Avito c ноября 2014,2016-08-16T18:09:34.606000
1,"Откачка септиков, выгребных ям и канализации в...",/solnechnogorsk/predlozheniya_uslug/otkachka_s...,,"Транспорт, перевозки . (Солнечн...",202463413,"Московская область, Солнечногорск",Осуществляем откачку септиков по Солнечногорск...,"Откачка септиков, выгребных ям и канализации",Размещено сегодня в 11:43.,Не указана,. (компания) на Avito c марта 2010,2016-08-16T18:09:35.341000
2,Москва Область Борт 20 тонн в Щелково,/schelkovo/predlozheniya_uslug/moskva_oblast_b...,,"Транспорт, перевозки Собственник ...",500571398,"Московская область, Щёлково",Грузоперевозки по Москве и Московской областиБ...,Москва Область Борт 20 тонн,Размещено сегодня в 08:57.,Не указана,Собственник (компания) на Avito c декабря 2014,2016-08-16T18:09:36.084000
3,Илосос для откачки бентонита (гнб шлам) в Голи...,/golitsyno/predlozheniya_uslug/ilosos_dlya_otk...,800 руб.,"Транспорт, перевозки Частное лицо ...",733885377,"Московская область, Голицыно",Вывозим буровой шлам ГНБ(бентонит)–есть ИЛОСОС...,Илосос для откачки бентонита (гнб шлам),Размещено вчера в 21:33.,800 руб.,Арсен Сеникович (частное лицо) на Avito c янв...,2016-08-16T18:09:36.863000
4,Откачка выгребных ям. Септиков в Одинцово,/odintsovo/predlozheniya_uslug/otkachka_vygreb...,,"Транспорт, перевозки Частное лицо ...",709651661,"Московская область, Одинцово","-Откачка воды, туалетов, сливных ям, автомоек ...",Откачка выгребных ям. Септиков,Размещено вчера в 09:03.,Не указана,Евгения (частное лицо) на Avito c января 2013,2016-08-16T18:09:37.387000
5,Откачка канализации в Некрасовском,/nekrasovskiy/predlozheniya_uslug/otkachka_kan...,1 300 руб.,"Транспорт, перевозки Частное лицо ...",657921227,"Московская область, Некрасовский","Быстро и недорого откачаем септик, выгребную я...",Откачка канализации,Размещено 14 августа в 22:56.,1 300 руб.,Роман (частное лицо) на Avito c ноября 2014,2016-08-16T18:09:37.902000
6,"Откачка выгребных ям, септиков, канализации в ...",/lobnya/predlozheniya_uslug/otkachka_vygrebnyh...,1 500 руб.,"Транспорт, перевозки Частное лицо ...",657931762,"Московская область, Лобня",Откачаем канализацию.Цена за 4 куб.мДлина шлан...,"Откачка выгребных ям, септиков, канализации",Размещено 14 августа в 22:55.,1 500 руб.,Роман (частное лицо) на Avito c ноября 2014,2016-08-16T18:09:38.464000
7,Откачка каналезации септиков биотуалетов истра...,/istra/predlozheniya_uslug/otkachka_kanalezats...,2 000 руб.,"Транспорт, перевозки Частное лицо ...",822228171,"Московская область, Истра",Откачка каналезации септиков биотуалетов истра...,Откачка каналезации септиков биотуалетов истра...,Размещено 14 августа в 22:39.,2 000 руб.,Сергей (частное лицо) на Avito c ноября 2013,2016-08-16T18:09:39.009000
8,"Откачка септиков, выгребных ям, туалетов звони...",/golitsyno/predlozheniya_uslug/otkachka_septik...,,"Транспорт, перевозки Частное лицо ...",672220227,"Московская область, Голицыно","Откачка, Голицыно, Малые Вязёмы, Жаворонки, Од...","Откачка септиков, выгребных ям, туалетов звоните",Размещено 14 августа в 20:16.,Не указана,Александр (частное лицо) на Avito c марта 2013,2016-08-16T18:09:39.549000
9,Откачка канализации септиков в Электроуглях,/elektrougli/predlozheniya_uslug/otkachka_kana...,1 000 руб.,"Транспорт, перевозки Валерий (Э...",780412666,"Московская область, Электроугли",Асенизаторная машина,Откачка канализации септиков,Размещено 14 августа в 17:15.,1 000 руб.,Валерий (компания) на Avito c марта 2016,2016-08-16T18:09:40.093000
